<a href="https://colab.research.google.com/github/birajsubhraguha/first_test/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import itertools
import numpy as np 
import pandas as pd
import collections as cl
import itertools as it
import scipy as sc
import random

In [4]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.5MB/s 


In [8]:
link='https://drive.google.com/open?id=1GXRSM6QHrxCGwYgOVIUOPXK9s-_fslas'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('AllVar.phot.gz')

import gzip
with gzip.open('AllVar.phot.gz', 'rt') as f:
    temp = f.read()
temp2 = temp.split("\n")
del temp2[-1]

dataa = [[float(y) for y in x.split(',')] for x in temp2]

1GXRSM6QHrxCGwYgOVIUOPXK9s-_fslas


In [0]:
df=pd.DataFrame(dataa,columns=['ID','time','mag','magerr','v5','v6'])
df=df.loc[:,'ID':'magerr']

import copy

# this makes a list of lists, where each list's 1st element is the key(ID), and the second element is a dataframe
df_grouped=list(df.groupby(by='ID'))
df_grouped=[list(i) for i in df_grouped]

# this groups w.r.t the stars' ID's
df_sorted=copy.deepcopy(df_grouped)
for i in range(len(df_sorted)):
    df_sorted[i][1]=df_sorted[i][1].sort_values(by='time')

# this reads in the ID table with labels and creates a dictionary
new=pd.read_csv('http://nesssi.cacr.caltech.edu/DataRelease/CatalinaVars.tbl',delim_whitespace=True)
df2=new.loc[:,['Catalina_Surveys_ID','Number_Obs']]
df2=df2.rename(columns={'Catalina_Surveys_ID':'ID','Number_Obs':'label'});
dic_label={df2['ID'][i]:df2['label'][i] for i in range(len(df2))}

# this creates a new df with ID's replaced by labels
df_labeled_sorted=copy.deepcopy(df_sorted)
for j in range(len(df_labeled_sorted)):
    temp=int(df_labeled_sorted[j][0])
    df_labeled_sorted[j]=[dic_label[temp]]+df_labeled_sorted[j]

In [10]:
sorted(cl.Counter(list(it.chain([i[0] for i in df_labeled_sorted]))).items(),key=lambda x:x[0])

[(1, 30593),
 (2, 4658),
 (3, 279),
 (4, 2420),
 (5, 5433),
 (6, 500),
 (7, 72),
 (8, 1514),
 (9, 62),
 (10, 124),
 (11, 242),
 (12, 7),
 (13, 512),
 (14, 142),
 (15, 25),
 (16, 85),
 (17, 153)]

df_big is a list of 17 elements, each corresponding to a class

Each element of df_big is a list of stars in that class, and has the format list(class, star_ID, time_series)

time_series is a pandas dataframe, with columns = (ID, time, mag, magerr)

In [0]:
#put proportion of test-train division here
prop=.1
tmp=sorted(cl.Counter(list(it.chain([i[0] for i in df_labeled_sorted]))).items(),key=lambda x:x[0])
freq=prop*np.array([a[1] for a in tmp])
freq=[int(i) for i in list(np.ceil(freq))]

#big dataset before division
df_big=[[list(i) for i in g] for k,g in it.groupby(sorted(df_labeled_sorted,key=lambda x:x[0]),key=lambda x:x[0])]

#test-train division
df_test=list()
df_train=list()
for i in range(len(df_big)):
    rng=range(len(df_big[i]))
    ind_test=random.sample(rng,freq[i])
    ind_train=list(set(rng)-set(ind_test))
    df_test.append([df_big[i][j] for j in ind_test])
    df_train.append([df_big[i][j] for j in ind_train])